In [ ]:
import ipywidgets as widgets

In [ ]:
lcc = widgets.Label(value="Current c++ class:")
tcc = widgets.Text()

l = widgets.Label(value="Input:")
ta = widgets.Textarea(layout=widgets.Layout(width='80%', height='200px'))
b = widgets.Button(description='Convert',tooltip='Convert',icon='check')

ol = widgets.Label(value="Output:")

olh = widgets.Label(value="Header:")
otah = widgets.Output(layout={'border': '1px solid black'})

ols = widgets.Label(value="Source:")
otas = widgets.Output(layout={'border': '1px solid black'})

ool = widgets.Label(value="Output:")
out = widgets.Output(layout={'border': '1px solid black'})

vbi = widgets.VBox([ lcc, tcc, l, ta, b])
vbo = widgets.VBox([ol, olh, otah, ols, otas, ool, out])


In [ ]:
if 'transform_input' in globals():
    b.on_click(transform_input, True)

class_remaps = {
    '': '',
    'Error': 'pandemonium_error',
    'void': 'void',
    'CharType': 'pandemonium_char_type',
    'Variant': 'pandemonium_variant',
    'bool': 'pandemonium_bool',
    'char': 'char',
    'int': 'pandemonium_int',
    'int32_t': 'int32_t',
    'int64_t': 'int64_t',
    'uint32_t': 'uint32_t',
    'uint64_t': 'uint64_t',
    'real_t': 'pandemonium_real',
    'float': 'pandemonium_real',
    'double': 'pandemonium_real',
    'String': 'pandemonium_string',
    'Rect2': 'pandemonium_rect2',
    'Rect2i': 'pandemonium_rect2i',
    'Vector2': 'pandemonium_vector2',
    'Vector2i': 'pandemonium_vector2i',
    'Vector3': 'pandemonium_vector3',
    'Vector3i': 'pandemonium_vector3i',
    'Vector4': 'pandemonium_vector4',
    'Vector4i': 'pandemonium_vector4i',
    'Plane': 'pandemonium_plane',
    'Quaternion': 'pandemonium_quaternion',
    'AABB': 'pandemonium_aabb',
    'Basis': 'pandemonium_basis',
    'Transform': 'pandemonium_transform',
    'Transform2D': 'pandemonium_transform2d',
    'Projection': 'pandemonium_projection',
    'Color': 'pandemonium_color',
    'NodePath': 'pandemonium_node_path',
    'RID': 'pandemonium_rid',
    'Object': 'pandemonium_object',
    'StringName': 'pandemonium_string_name',
    'Dictionary': 'pandemonium_dictionary',
    'Array': 'pandemonium_array',
    'PoolByteArray': 'pandemonium_pool_byte_array',
    'PoolIntArray': 'pandemonium_pool_int_array',
    'PoolRealArray': 'pandemonium_pool_real_array',
    'PoolStringArray': 'pandemonium_pool_string_array',
    'PoolVector2Array': 'pandemonium_pool_vector2_array',
    'PoolVector2iArray': 'pandemonium_pool_vector2i_array',
    'PoolVector3Array': 'pandemonium_pool_vector3_array',
    'PoolVector3iArray': 'pandemonium_pool_vector3i_array',
    'PoolVector4Array': 'pandemonium_pool_vector4_array',
    'PoolVector4iArray': 'pandemonium_pool_vector4i_array',
    'PoolColorArray': 'pandemonium_pool_color_array',
    'CharString': 'pandemonium_char_string',
    'Char16String': 'pandemonium_char_16_string',
}

class_c_no_force_ptr = {
    'bool': True,
    'int': True,
    'real_t': True,
    'float': True,
    'double': True,
    'int32_t': True,
    'int64_t': True,
    'uint32_t': True,
    'uint64_t': True,
    'Error': True,
}

METHOD_ARG_TYPE_NONE = 0
METHOD_ARG_TYPE_REFERENCE = 1
METHOD_ARG_TYPE_POINTER = 2

def get_current_cpp_class():
    return tcc.value

def get_current_c_class():
    return class_remaps[tcc.value]
    

class PMethodArg:
    def __init__(self):
        self.is_const = False
        self.cpp_type = ''
        self.cpp_arg_type = METHOD_ARG_TYPE_NONE
        self.cpp_has_default = False
        self.arg_name = ''
        self.cpp_default_value = ''
        
    def is_no_force_ptr(self):
        return self.cpp_type in class_c_no_force_ptr
    
    def get_arg_name_local(self):
        return self.arg_name[2:]
    
    def generate_c_arg(self):
        s = ', '
        
        if self.is_const:
            s += 'const '
            
        s += class_remaps[self.cpp_type] + ' '
        
        is_pointer = self.cpp_arg_type != METHOD_ARG_TYPE_NONE
        
        if is_pointer and self.cpp_type in class_c_no_force_ptr:
            is_pointer = False
            
        if is_pointer:
            s += '*'
        
        s += self.arg_name
        
        return s
    
    def generate_convert_line(self):
        s = ''

        if self.is_const:
            s += 'const '
        
        if self.is_no_force_ptr():
            s += self.cpp_type + ' ' + self.get_arg_name_local() + ' = ' + self.arg_name + ';'
        else:
            s += self.cpp_type + ' *' + self.get_arg_name_local() + ' = ('
            
            if self.is_const:
                s += 'const '
                
            s += self.cpp_type + ' *)' + self.arg_name + ';'
             
        return s
    
    def parse_line(self, l):
        self.cpp_has_default = False
        
        if '=' in l:
            sl = l.split('=')
            
            l = sl[0].strip()
            self.cpp_has_default = True
            self.cpp_default_value = sl[1].strip()
        
        
        self.cpp_arg_type = METHOD_ARG_TYPE_NONE
        
        if '*' in l:
            self.cpp_arg_type = METHOD_ARG_TYPE_POINTER
            
        if '&' in l:
            self.cpp_arg_type = METHOD_ARG_TYPE_REFERENCE
            
        l = l.replace('*', '')
        l = l.replace('&', '')
        
        spls = l.split(' ')
        
        self.is_const = spls[0].strip() == 'const'
        
        if self.is_const:
            spls = spls[1:]
        
        with out:
            print(spls)
        
        self.cpp_type = spls[0]
        self.arg_name = spls[1]
    
    def __str__(self):
        s = '    PMethodArg:\n'
        
        s += '      is_const: ' + str(self.is_const) + '\n'
        s += '      cpp_type: ' + self.cpp_type + '\n'
        s += '      cpp_arg_type: ' + str(self.cpp_arg_type) + '\n'
        s += '      arg_name: ' + self.arg_name + '\n'
        s += '      cpp_has_default: ' + str(self.cpp_has_default) + '\n'
        s += '      cpp_default_value: ' + self.cpp_default_value + '\n'
        
        return s
    


class PMethod:
    def __init__(self):
        self.args = []
    
        self.method_name = ''

        self.cpp_return_type = ''
        self.is_const = False
        self.is_constructor = False
    
    def parse_line(self, l):
        l = l.replace('//', '').strip()
        l = l.replace(';', '').strip()
        l = l.replace('inline', '').strip()
        l = l.replace('_FORCE_INLINE_', '').strip()
        l = l.strip()
        
        self.is_const = False
        
        if l.endswith('const'):
            self.is_const = True
            
            #remove it from the string
            
            l = l[:-5]
            
        l = l.replace(')', '')
        
        sps = l.split('(')
        
        mdeffull = sps[0].strip()
        margsfull = sps[1].strip()
        
        mds = mdeffull.split(' ')
        
        self.is_constructor = False
        
        mrettype = ''
        mdef = ''
        
        if len(mds) == 1:
            self.is_constructor = True
            self.method_name = mds[0].strip()
        else:
            self.cpp_return_type = mds[0].strip()
            self.method_name = mds[1].strip()
            
        margsspl = margsfull.split(',')
        
        for ml in margsspl:
            ml = ml.strip()
            
            if ml == '':
                continue
            
            a = PMethodArg()
            a.parse_line(ml)
            self.args.append(a)
    
    def get_c_return_type(self):
        return class_remaps[self.cpp_return_type]
    
    def get_c_method_name(self):
        return get_current_c_class() + '_' + self.method_name

    def generate_method_string(self):
        s = ''
        
        s += self.get_c_return_type() + ' GDAPI ' + self.get_c_method_name() + '('
        
        if self.is_const:
            s += 'const '
            
        s += get_current_c_class() + ' *p_self'
        
        for a in self.args:
            s += a.generate_c_arg()
        
        s += ')'
        
        return s
    
    def generate_header_string(self):
        return self.generate_method_string() + ';'
    
    def generate_impl_string(self):
        s = self.generate_method_string() + '{\n'
        
        # Create dest variable at top if needed
        return_needs_conversion = False
        
        if self.cpp_return_type != 'void':
            if not self.cpp_return_type in class_c_no_force_ptr:
                return_needs_conversion = True
        
        if return_needs_conversion:
            # it needs to be explicitly converted
            # Just Add dest var here
            s += '\t' + self.get_c_return_type() + ' dest;\n'
            
        s += '\t'
        
        # Create self local
        if self.is_const:
            s += 'const '
            
        s += get_current_cpp_class() + '* self = ('
        
        if self.is_const:
            s += 'const '
            
        s += get_current_cpp_class() + ' *)p_self;\n'
            
        # Create param locals
        self_call_args = []
        
        for a in self.args:
            if a.is_no_force_ptr():
                self_call_args.append(a.arg_name)
            else:
                self_call_args.append('*' + a.get_arg_name_local())
                
                s += '\t' + a.generate_convert_line() + '\n'
    
        # Create method call
        s += '\t'

        # We have a return type
        if self.cpp_return_type != 'void':
            if return_needs_conversion:
                s += '*((' + self.cpp_return_type + ' *)&dest) = '
            else:
                s += 'return '

        s += 'self->' + self.method_name + '('
            
        first = True
        for a in self_call_args:
            if first:
                first = False
                s += a
            else:
                s += ', ' + a
                
        s += ');\n'
    
        if self.cpp_return_type != 'void':
            if return_needs_conversion:
                s += '\treturn dest;\n'
    
        s += '}'
        
        return s
    
    def __str__(self):
        s = 'PMethod:\n'
        
        s += '  method_name: ' + self.method_name + '\n'
        s += '  cpp_return_type: ' + self.cpp_return_type + '\n'
        s += '  is_const: ' + str(self.is_const) + '\n'
        s += '  is_constructor: ' + str(self.is_constructor) + '\n'
        
        s += '  args: [\n'
        
        for a in self.args:
            s += str(a)
            s += '\n'
            
        s += '  ]'
            
        return s

def transform_input(b):
    with out:
        out.clear_output()
        
    with otah:
        otah.clear_output()
        
    with otas:
        otas.clear_output()
        
    lines = ta.value.split('\n')
    
    for l in lines:
        if l.strip() == '':
            with otah:
                print('')
                
            with otas:
                print('')
                   
            continue
            
        with out:
            print('line:')
            print(l)
            print('')
            
        method = PMethod()
        method.parse_line(l)
        
        with otah:
            print(method.generate_header_string())
            
        with otas:
            print(method.generate_impl_string())
        
        with out:
            print(str(method))


b.on_click(transform_input)

# Input

In [ ]:
display(vbi)

# Output

In [ ]:
display(vbo)